# Test sample and feature weight

This notebook explores whether weighting each sample with the number of events improve the accuracy of the prediction? 
  * What is an appropiate weight for devices without events?
  * Should the weight be capped? 
  * How should it increase with number of events?
  
Unfortunately, various possible weighting scheme (normalized, linear, exponential) all decreased the score. I do not understand this, requires further work and better understanding the the lbfgs solver.

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import time
import sys

#import sys
#import pymongo
#import bson.objectid
#pymongo.objectid = bson.objectid
#sys.modules["pymongo.objectid"] = bson.objectid

#os.environ['KERAS_BACKEND']='tensorflow'
os.environ['THEANO_FLAGS'] = 'device=cpu'
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.callbacks import Callback
import keras

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.cross_validation import StratifiedKFold, train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
import pickle

Using Theano backend.


In [6]:
dir_in = 'data_ori'
dir_feat = 'data'
rs = 123
fixed_seed_num = 1234
np.random.seed(fixed_seed_num)
random.seed(fixed_seed_num) # not sure if needed or not

feature_files_old = ['features_bag_brand',
                 'features_bag_model',
                'features_bag_label_installed',
                'features_bag_app_installed']
feature_files = ['features_brand_bag',
                 'features_brand_model_bag',
                 'features_brand_model.csv']
feature_files=feature_files[0:2]

# Function to read feature file
def open_feature_file(fname, samples='train'):
    if fname[-3:] == 'csv':
        if samples=='train':
            X = gatrain[['device_id']].merge( pd.read_csv(os.path.join(dir_feat, fname)), on='device_id', how='left')
        else:
            X = gatest[['device_id']].merge( pd.read_csv(os.path.join(dir_feat, fname)), on='device_id', how='left')
            
        X.drop('device_id', axis=1, inplace=True)
        X.fillna(0, inplace=True)
        return csr_matrix(X.values)
    else:
        # Assume it is a pickle file
        with open(os.path.join(dir_feat, '{}_{}.pickle'.format(fname,samples)), 'rb') as f:
            return pickle.load(f)

In [7]:
# Calculate CV score on Xtrain, y
def score(X, clf = None, random_state = 123, w = None):
    if not clf: clf = LogisticRegression(C=0.02, multi_class='multinomial',solver='lbfgs', n_jobs=2)
        
    kf = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=random_state)
    pred = np.zeros((y.shape[0],nclasses))
    for itrain, itest in kf:
        Xtr, Xte = X[itrain, :], X[itest, :]
        ytr, yte = y[itrain], y[itest]
        if np.any(w):
            ws = w[itrain]
        else:
            ws = None
            
        clf.fit(Xtr, ytr, sample_weight = ws)
        pred[itest,:] = clf.predict_proba(Xte)
        # Downsize to one fold only for kernels
        #print("{:.5f}".format(log_loss(yte, pred[itest,:])))
    #print('')
    return log_loss(y, pred)

## Get basescore on installed bag of apps

In [8]:
gatrain = pd.read_csv(os.path.join(dir_in,'gender_age_train.csv'))
gatest = pd.read_csv(os.path.join(dir_in,'gender_age_test.csv'))
targetencoder = LabelEncoder().fit(gatrain.group)
y = targetencoder.transform(gatrain.group)
nclasses = len(targetencoder.classes_)

In [9]:
fw = [1, 1, 1, 1]
Xtrain = hstack([open_feature_file(f) for f in feature_files],format='csr')
# Weird linear scaling of features decreases performance.
# *1.0: Basescore: 2.27402
# *1.1: Basescore: 2.27443
# *1.2: Basescore: 2.27534
# *0.5: Basescore: 2.28733
Xtest = hstack([open_feature_file(f,'test') for f in feature_files],format='csr')

In [10]:
print('Training on {} features'.format(Xtrain.shape[1]))
basescore = score(Xtrain, random_state=rs)
print('Basescore: {:.5f}'.format(basescore))

Training on 1798 features
Basescore: 2.39530


## Test Keras model

In [11]:
dummy_y = np_utils.to_categorical(y)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(Xtrain, dummy_y, 
                                                  test_size=0.1, random_state=42)

In [13]:
def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch, np.ones(X_batch.shape[0])
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch#, np.ones(X_batch.shape[0])
        if (counter == number_of_batches):
            counter = 0

# define baseline model
params = {'optimizer': 'adamax',
          'batch_size': hp.uniform('batch_size', 16, 512),
          'n_epoch': 3,
         'layer_1': {'on1': True,
                     'units1': 100,
                     'activation1': 'relu',
                     'dropout1': 0.2},
         'layer_2': {'on2': True,
                    'units2': 150,
                    'activation2': 'tanh',
                    'dropout2': 0.2},
          'layer_3': {'on3': False,
                     'units3': 12,
                     'activation3': 'sigmoid',
                     'dropout3': 0.2}
          }

# define baseline model
params = {'optimizer': 'adamax',
          'batch_size': 64, #hp.uniform('batch_size', 16, 512),
          'n_epoch': hp.uniform('n_epoch', 8, 64),
         'layer_1': {'on1': True,
                     'units1': hp.uniform('units1', 64, 1024),
                     'activation1': hp.choice('activation1',['relu','tanh']),
                     'dropout1': hp.uniform('dropout1', .25, .75)},
         'layer_2': {'on2': hp.choice('on2',[True, False]),
                    'units2': hp.uniform('units2', 64, 1024),
                    'activation2': hp.choice('activation2',['relu','tanh']),
                    'dropout2': hp.uniform('dropout2', .25, .75)},
          'layer_3': {'on3': False,
                     'units3': hp.uniform('units3', 64, 1024),
                     'activation3': hp.choice('activation3',['relu','tanh']),
                     'dropout3': hp.uniform('dropout3', .25, .75)}
          }


def f_nn(params):
    # create model
    print ('Params testing: ', params)
    
    model = Sequential()
    model.add(Dense(params['layer_1']['units1'], 
                    input_dim=Xtrain.shape[1], 
                    init='normal', 
                    activation=params['layer_1']['activation1']))
    
    model.add(Dropout(params['layer_1']['dropout1']))
    
    if params['layer_2']['on2']:
        model.add(Dense(params['layer_2']['units2'], 
                        input_dim=Xtrain.shape[1], 
                        init='normal', 
                        activation=params['layer_2']['activation2']))
        #https://www.kaggle.com/poiss0nriot/talkingdata-mobile-user-demographics/bag-of-apps-keras-11-08-16-no-val/run/328610
        #model.add(PReLU())
        model.add(Dropout(params['layer_2']['dropout2']))
    
    if params['layer_3']['on3']:
        model.add(Dense(params['layer_3']['units3'], 
                        init='normal', 
                        activation=params['layer_3']['activation3']))
        
        model.add(Dropout(params['layer_3']['dropout3']))
    
    model.add(Dense(12, init='normal', activation='softmax'))    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])  #logloss
    
    hist_all = prediction_hist('test')
    
    fit=model.fit(X_train.todense(), y_train, nb_epoch = int(params['n_epoch']), batch_size=int(params['batch_size']),
                             validation_data=(X_val.todense(), y_val), verbose=2,
                 callbacks=[hist_all]) 
    
    scores_val = model.predict_proba(X_val.todense(), batch_size = 128, verbose = 0)

    logloss = log_loss(y_val, scores_val)
    print('Log_loss:', logloss)
    sys.stdout.flush() 
    return {'loss': logloss, 'status': STATUS_OK}

def plot_loss_progress(pred_hist_obj, ax1=None, ax2=None, c='g'):
    
    df = pred_hist_obj.predhist
    
    if not ax1:
        fig, (ax1, ax2) = plt.subplot(1, 2, figsize=(16,4))
        
    df.plot(y='acc', ax=ax1, label=pred_hist_obj.tag, color=c)
    df.plot(y='val_acc', ax=ax1, label=pred_hist_obj.tag, linestyle='--', color=c)
    ax1.set_title('Accuracy score')
    ax1.set_xlabel('Epoch #')
    ax1.set_ylabel('Accuracy')
    
    df.plot(y='logloss', ax=ax2, label=pred_hist_obj.tag, color=c)
    df.plot(y='val_logloss', ax=ax2, label=pred_hist_obj.tag, linestyle='--', color=c)
    ax2.set_title('Logloss score')
    ax2.set_xlabel('Epoch #')
    ax2.set_ylabel('Logloss')

class prediction_hist(Callback):
    def __init__(self, tag=''):
        self.predhist = pd.DataFrame(columns=['acc','logloss','val_acc','val_logloss'])
        self.tag = str(tag)
        
    def on_epoch_end(self, epoch, logs={}):
        #print logs.values()
        self.predhist.loc[epoch] = logs.values()
        

In [ ]:
trials = Trials()
best = fmin(f_nn, params, algo=tpe.suggest, max_evals=1000, trials=trials)
print 'best: '
print best


('Params testing: ', {'n_epoch': 48.27241736866729, 'optimizer': 'adamax', 'layer_1': {'units1': 281.777395421635, 'activation1': 'tanh', 'on1': True, 'dropout1': 0.45546218635450947}, 'layer_3': {'activation3': 'tanh', 'on3': False, 'units3': 732.6104181739472, 'dropout3': 0.49555946671629864}, 'layer_2': {'on2': True, 'activation2': 'relu', 'units2': 338.69376155236426, 'dropout2': 0.6400138809560396}, 'batch_size': 64})
Train on 67180 samples, validate on 7465 samples
Epoch 1/48
10s - loss: 2.4127 - acc: 0.1411 - val_loss: 2.4043 - val_acc: 0.1440
Epoch 2/48
15s - loss: 2.3988 - acc: 0.1505 - val_loss: 2.3995 - val_acc: 0.1449
Epoch 3/48
16s - loss: 2.3918 - acc: 0.1554 - val_loss: 2.3977 - val_acc: 0.1480
Epoch 4/48
17s - loss: 2.3864 - acc: 0.1571 - val_loss: 2.3975 - val_acc: 0.1480
Epoch 5/48
17s - loss: 2.3812 - acc: 0.1613 - val_loss: 2.3987 - val_acc: 0.1449
Epoch 6/48
17s - loss: 2.3778 - acc: 0.1625 - val_loss: 2.3986 - val_acc: 0.1457
Epoch 7/48
17s - loss: 2.3746 - acc: 0

In [ ]:
parameters = ['batch_size']
f, axes = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
cmap = plt.cm.jet
for i, val in enumerate(parameters):
    print i, val
    xs = np.array([t['misc']['vals'][val] for t in trials.trials]).ravel()
    ys = [-t['result']['loss'] for t in trials.trials]
    #xs, ys = zip(sorted(xs), sorted(ys))
    #ys = np.array(ys)
    axes[i/3,i%3].scatter(xs, ys, s=20, linewidth=0.01, alpha=0.5, c=cmap(float(i)/len(parameters)))
    axes[i/3,i%3].set_title(val)

In [ ]:



fig, (ax1, ax2) = plt.subplots(1,2,figsize=(16,4))
drs = [0.1, 0.2, 0.3, 0.4, 0.5]
optim = ['adamax', 'adam', 'nadam', 'adadelta']
cs = 'grbycm'
for i,dr in enumerate(optim):
    hist_all = prediction_hist(tag=optim)
    
    model=baseline_model(params)
    
    fit=model.fit(X_train.todense(), y_train, nb_epoch = 7, batch_size=32,
                             validation_data=(X_val.todense(), y_val), verbose=2,
                 callbacks=[hist_all]) 
    
    plot_loss_progress(hist_all, ax1=ax1, ax2=ax2, c=cs[i])

In [ ]:
model.summary()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(16,4))

cs = 'grbycm'
plot_loss_progress(hist_all, ax1=ax1, ax2=ax2, c=cs[i])

In [ ]:
print hist_generator.predhist.head()
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(16,4))
plot_loss_progress(hist_all, ax1=ax1, ax2=ax2)

In [ ]:
# evaluate the model
scores_val = model.predict_generator(generator=batch_generatorp(X_val, 32, False), val_samples=X_val.shape[0])
scores = model.predict_generator(generator=batch_generatorp(Xtest, 32, False), val_samples=Xtest.shape[0])

print('logloss val {}'.format(log_loss(y_val, scores_val)))

In [ ]:
basescore = 2.27506319419 (2.2751433925) with weight in batch_generator: 2.2751433925, 2.2751433925

In [ ]:
pred = pd.DataFrame(scores, index = gatest['device_id'], columns=targetencoder.classes_)
pred.to_csv('A_keras_model_on_apps_and_labels.csv',index=True)

## Fit on complete model

In [ ]:
model=baseline_model()
fit=model.fit(X_train.todense(), y_train, nb_epoch = 20, batch_size=32,
                         validation_data=(X_val.todense(), y_val), verbose=2)

In [ ]:
# evaluate the model
scores_val = model.predict_generator(generator=batch_generatorp(X_val, 32, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val)))

In [ ]:
model.summary()